# Migrations résidentielles en 2017 : lieu de résidence actuelle - lieu de résidence antérieure

Source : https://www.insee.fr/fr/statistiques/4509335


In [1]:
import pandas as pd

In [56]:
flux = pd.read_csv('../data/insee/base-flux-mobilite-residentielle-2017.csv', sep=";", dtype={'CODGEO': str, 'DCRAN': str})

In [57]:
len(flux)

477803

In [66]:
big_communes = (flux
    [
        (flux['CODGEO'] == flux['DCRAN'])
        & (flux['DCRAN'].str[0:2] != '97')
        & (flux['CODGEO'].str[0:2] != '97')
    ]
    .query('NBFLUX_C17_POP01P >= 50000')
    [['CODGEO']]
    .drop_duplicates(['CODGEO'])
)

In [67]:
big_flux = (flux
    [
        (flux['CODGEO'].isin(big_communes['CODGEO'])) 
        & (flux['DCRAN'].isin(big_communes['CODGEO']))
        & (flux['DCRAN'] != flux['CODGEO'])        
    ]
    .rename(columns={
        'DCRAN': 'input',
        'CODGEO': 'output',
        'NBFLUX_C17_POP01P': 'count'
    })
    [['input','output','count']]    
)

big_flux.to_csv('../public/data/flux.csv', sep=';', index=False, float_format="%.0f")

In [20]:
import geopandas as gpd

In [75]:
coms = gpd.read_file('../data/ign/COMMUNE.shp')
(coms
    [coms['INSEE_COM'].isin(big_communes['CODGEO'])]
    [['INSEE_COM','geometry']]    
    .assign(geometry = lambda df: df['geometry'].centroid)
    .to_file('../public/data/communes.geo.json', driver='GeoJSON')
)

In [74]:
coms.sample(3)['geometry'].centroid

14044    POINT (5.96625 46.83345)
31443    POINT (5.43229 45.31399)
23064    POINT (1.62139 43.49846)
dtype: geometry